In [30]:
from copy import deepcopy

f = open("data.txt", "r")
data = f.read()
rules, messages = data.split('\n\n')
rules = rules.split('\n')
messages = messages.split('\n')
# data = [x.strip('\n') for x in data]



In [31]:
# Part 1 Monster Messages
# why use re when I can just make DFAs myself?????

def parseRules(rules):
    table = {}

    for rule in rules:
        state, transFunction = rule.split(': ')
        # the transition function can be a letter, 1 or 2 states, or between a set of two states
        if '\"' in transFunction: # a letter
            table[state] = transFunction.strip('\"')
        else:
            # table[state] = transFunction.split(' | ')

            table[state] = [x.split(' ') for x in transFunction.split(' | ')]
        


        # print(state)
        # print(transFunction)
    return table
table = parseRules(rules)
table['0']

[['8', '11']]

In [32]:
# string: aaabbbb
# rule: 1 2
# how do we know where rule 1 ends??
# we can expand the range we check until it works or doesnt


In [33]:
# message is a substring of orginal message
# partialRule is just a single rule
def checkPartialRule(message, partialRule, table):
    print(checkPartialRule.__name__, f'message={message}, partialRule={partialRule}')
    # get the rule for partialRule
    rule = table[partialRule]
    print(f'\trule={rule}')
    # print(rule)
    if len(rule) == 1 and rule[0][0].isalpha():
        print(f'returns {message == rule}')
        return message == rule
    else:
        print(f'returns checkWholeRule')
        return checkWholeRule(message, rule, table)

In [34]:
def checkWholeRule(message, rules, table):
    print(checkWholeRule.__name__, f'message={message}, rules={rules}')

    for rule in rules:
        # keep track of the last index of each partial rule
        if len(rule) > len(message):
            print('\tFALSE: len(rule) > len(message)')
            return False
        # ruleWindows = list(reversed(range(len(rule))))
        # ruleWindows = list(map(lambda x: len(message) - x, ruleWindows))
        # calcWindowSizes(ruleWindows)

        ruleWindows = list(range(len(rule)))
        ruleWindows[-1] = len(message)
        ruleWindows = list(map(lambda x: x+1, ruleWindows))
        for currRuleWindow in calcWindowSizes(ruleWindows):
            print(checkWholeRule.__name__, f'message={message}, rules={rules}')
            print(f'\tcurrRuleWindow={currRuleWindow}')
            # check if the current rule windows are valid
            prevRW = 0
            validRules = True
            for rw, r in zip(currRuleWindow, rule):
                # print(f'\trw={rw}, r={r}')
                if not checkPartialRule(message[prevRW:rw], r, table):
                    validRules = False
                    break
                prevRW = rw
            if validRules:
                return True

            # at this point we know that the current rule windows are not valid and they need to be adjusted
            # we want to shrink the first rule, then test all combinations of the other rules if there are any
            # so we have to keep track of how many adjustments we made for all the steps
            # aabc
            # abbc
            # abcc
        # print(rule)

In [35]:
def checkMessages(messages, table):
    count = 0
    a = len(messages)
    b = 0
    for message in messages:
        if checkWholeRule(message, table['0'], table):
            count += 1
        b += 1
        print(f'{b}/{a}')
    print(count)

In [36]:
# damn no good tools to do this
# guess we gotta write out own
def calcWindowSizes(ruleWindows):
    rw = ruleWindows[:]
    currRW = ruleWindows[:]
    # we dont know how many elements are in ruleWindows by default this
    # the first time we want to yield the currrent ruleWindows
    yield rw
    if len(rw) > 1:
        for i in range(1,rw[-1]-len(rw)+2):
            currRW[0] = i
            yield from calcWindowSizesHelper(currRW, ruleWindows[1:], 1)


def calcWindowSizesHelper(currRW, ruleWindows, index):
    # print(ruleWindows)
    # print("two", ruleWindows[0])
    if len(ruleWindows) > 1:
        curr = currRW[index-1]+1
        while curr < currRW[index+1]:
            currRW[index] = curr
            # currRW[index+1] = currRW[index+1] +1
            yield from calcWindowSizesHelper(currRW, ruleWindows[1:], index+1)
            curr += 1
    else:
        yield currRW


In [37]:
checkMessages(messages, table)

checkWholeRule message=aabbbbbabaaaaabbbbabaabb, rules=[['8', '11']]
	currRuleWindow=[1, 25]
checkPartialRule message=a, partialRule=8
	rule=[['42']]
returns checkWholeRule
checkWholeRule message=a, rules=[['42']]
	currRuleWindow=[2]
checkPartialRule message=a, partialRule=42
	rule=[['77', '36'], ['91', '69']]
returns checkWholeRule
checkWholeRule message=a, rules=[['77', '36'], ['91', '69']]
	FALSE: len(rule) > len(message)
	currRuleWindow=[1, 25]
checkPartialRule message=a, partialRule=8
	rule=[['42']]
returns checkWholeRule
checkWholeRule message=a, rules=[['42']]
	currRuleWindow=[2]
checkPartialRule message=a, partialRule=42
	rule=[['77', '36'], ['91', '69']]
returns checkWholeRule
checkWholeRule message=a, rules=[['77', '36'], ['91', '69']]
	FALSE: len(rule) > len(message)
	currRuleWindow=[2, 25]
checkPartialRule message=aa, partialRule=8
	rule=[['42']]
returns checkWholeRule
checkWholeRule message=aa, rules=[['42']]
	currRuleWindow=[3]
checkPartialRule message=aa, partialRule=42


KeyboardInterrupt: 

In [29]:
message = "abbb"
# ruleWindows = list(reversed(range(len(['77', '99']))))
ruleWindows = list(range(len(['77', '99'])))
ruleWindows[-1] = len(message)

ruleWindows = list(map(lambda x: x+1, ruleWindows))



for a in calcWindowSizes(ruleWindows):
    print(a)

[1, 5]
[1, 5]
[2, 5]
[3, 5]
[4, 5]
